# Gurobi Python API简介

###### Gurobi Days Digital
###### June 24, 2026
###### Maliheh Aramon, PhD, Optimization Engineer
###### aramon@gurobi.com

## 如何在本地运行笔记本？  
- 访问[Gurobi建模示例](https://github.com/Gurobi/modeling-examples)存储库
- 克隆包含本笔记本和其他示例的存储库，或者点击[这里](https://github.com/Gurobi/modeling-examples/archive/refs/heads/master.zip)下载
- 导航到 __intro_to_gurobipy__ 文件夹
- 启动[Jupyter Notebook Server](https://docs.jupyter.org/en/latest/running.html#id2)
- 在Jupyter notebook中打开笔记本
- 笔记本将安装gurobipy包和其他依赖项。Gurobi pip包包括一个大小有限的试用许可证，允许您运行笔记本

In [ ]:
%pip install "gurobipy>=10.0"

# 安装其他依赖项
%pip install numpy
%pip install scipy
%pip install pandas

# Gurobi Python API

Gurobi Python API，也称为gurobipy，是最流行的Gurobi API，因为它允许使用
- 单独的变量和约束条件，与Guorbi的其他面向对象的API（如 C、C++、Java 和 .NET）一样
- 矩阵，如其他Gurobi的面向矩阵的接口，如MATLAB和R
- 更详细的数学语法，如传统的建模语言

在本节课中，我们将带您了解Gurobi Python API的基础知识。

## 如何安装 gurobipy？
在任何操作系统（如Linux、Windows或macOS）上安装gurobipy主要有两种方法。

- Pip
- Conda

相关知识库（KB）文章：
- [Gurobi支持哪些Python版本？](https://support.gurobi.com/hc/en-us/articles/360013195212)
- [如何为Python安装Gurobi？](https://support.gurobi.com/hc/en-us/articles/360044290292-How-do-I-install-Gurobi-for-Python-)

## 优化模型

Gurobi可以处理的优化模型的规范形式如下：

\begin{align}
\mbox{Model P:} ~~~~ \mbox{minimize} \quad & x^T Q x + c^T x + d &  \notag \\
\mbox{subject to} \quad & Ax = b & \notag & \notag \\
                        & x^T Q_i x + c_i^T x \leq d_i & \forall i \in I \notag \\
                        & l \leq x \leq u & \notag \\
                        & x_j \in \mathbb{Z} & \forall j \in J 
\end{align}

在本节课中，我们将学习如何使用gurobipy将上述数学结构映射到代码中。

每个数学模型都有四个主要元素：__数据+决策变量+约束+目标函数__

__数据__:
\begin{align}
& \mbox{Sets:}~~ I, J & \notag \\ 
& \mbox{Coefficients:}~~ Q, c, A, Q_i, c_i \notag &\\ 
& \mbox{RHS values:}~~ b, d_i & \notag \\ 
& \mbox{Lower and upper bounds:}~~ l, u & \notag \\ 
& \mbox{Constants:}~~ d & \notag \\
& \mbox{Operators:} & \notag \\
& ~~~~~~ \mbox{Arithmetic}~ (+, -, *, \div)& \notag \\
& ~~~~~~ \mbox{Constraint operators} ~(\geq, \leq, =) & \notag
\end{align}

## 简单示例
让我们从一个简单的例子开始：

\begin{align}
\mbox{maximize} \quad & x + y + 2z \notag \\
\mbox{subject to} \quad & x + 2y + 3z \leq 4 \notag \\
                        & x + y \geq 1 \notag \\
                        & x, y, z \in \{0, 1\} \notag
\end{align}

In [ ]:
# 为方便起见，导入gurobipy包为gp
import gurobipy as gp

# GRB 是所有 Gurobi 常量的列表
from gurobipy import GRB

# 创建一个Gurobi环境和一个模型对象
with gp.Env() as env, gp.Model("simple-example", env=env) as model:
    # 定义决策变量
    x = model.addVar(vtype=GRB.BINARY, name="x")
    y = model.addVar(vtype=GRB.BINARY, name="y")
    z = model.addVar(vtype=GRB.BINARY, name="z")

    # 定义约束
    model.addConstr(x + 2 * y + 3 * z <= 4, name="c0")
    model.addConstr(x + y >= 1, name="c1")

    # 定义目标函数
    model.setObjective(x + y + 2 * z, sense=GRB.MAXIMIZE)

    # 求解模型
    model.optimize()

    print("******* Solution *******")
    for var in model.getVars():
        print(f"{var.VarName}: {var.X}")
    print("************************")

Set parameter LicenseID to value 2601452
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x98886187
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 18 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
******* Sol

## Python数据结构

- 元组：一种有序的复合分组，一旦创建就不能修改，它非常适合表示多维下标。
    ```
        ("city_0", "city_1")
    ```
- 列表：一个有序的组，因此每个项都有索引。可以通过添加、删除或排序元素来修改列表。
    ```
        ["city_0", "city_1", "city_2"]
     ```
- 集合：一组无序的唯一元素。只能通过添加或删除来修改集合。
    ```
        {"city_0", "city_1", "city_2"}
    ```
- 字典：键值对映射，非常适合表示成本、需求、容量等索引数据。
    ```
        damand = {"city_0": 100, "city_1": 50, "city_2": 40}
    ```

 ## Gurobi Python API中的扩展数据结构
 
 - [tuplelist()](https://www.gurobi.com/documentation/current/refman/py_tuplelist.html)
     - Python list的子类
     - 有效构建子列表的重要方法
         - [tuplelist.select(pattern)](https://www.gurobi.com/documentation/current/refman/py_tuplelist_select.html) --> tuplelist()
     
 
 
- [tupledict()](https://www.gurobi.com/documentation/current/refman/py_tupledict.html)
    - Python字典的子类
    - tupledict()的键以 tuplelist()的形式存储，值为 Gurobi 变量对象。
    - 有效构建线性表达式的重要方法:
        - [tupledict.select(pattern)](https://www.gurobi.com/documentation/current/refman/py_tupledict_select.html) --> List
        - [tupledict.sum(pattern)](https://www.gurobi.com/documentation/current/refman/py_tupledict_sum.html) --> LinExpr()
        - [tupledict.prod(coeff, pattern)](https://www.gurobi.com/documentation/current/refman/py_tupledict_prod.html) --> LinExpr()
    
    
- [multidict()](https://www.gurobi.com/documentation/current/refman/py_multidict.html): 在一个语句中定义多个字典的方便函数。

In [1]:
import gurobipy as gp
from gurobipy import GRB

data = gp.tupledict(
    [
        (("a", "b", "c"), 3),
        (("a", "c", "b"), 4),
        (("b", "a", "c"), 5),
        (("b", "c", "a"), 6),
        (("c", "a", "b"), 7),
        (("c", "b", "a"), 3),
    ]
)
print(f"data: {data}")

data: {('a', 'b', 'c'): 3, ('a', 'c', 'b'): 4, ('b', 'a', 'c'): 5, ('b', 'c', 'a'): 6, ('c', 'a', 'b'): 7, ('c', 'b', 'a'): 3}


In [2]:
print("\nTuplelist:")
keys = gp.tuplelist(data.keys())
print(f"\tselect: {keys.select('a', '*', '*')}")


Tuplelist:
	select: <gurobi.tuplelist (2 tuples, 3 values each):
 ( a , b , c )
 ( a , c , b )
>


In [3]:
print("\nTupledict:")
print(f"\tselect  : {data.select('a', '*', '*')}")
print(f"\tsum     : {data.sum('*', '*', '*')}")
coeff = {("a", "c", "b"): 6, ("b", "c", "a"): -4}
print(f"\tprod    : {data.prod(coeff, '*', 'c', '*')}")


Tupledict:
	select  : [3, 4]
	sum     : 28.0
	prod    : 0.0


In [4]:
arcs, capacity, cost = gp.multidict(
    {
        ("Detroit", "Boston"): [100, 7],
        ("Detroit", "New York"): [80, 5],
        ("Detroit", "Seattle"): [120, 4],
        ("Denver", "Boston"): [120, 8],
        ("Denver", "New York"): [120, 11],
        ("Denver", "Seattle"): [120, 4],
    }
)
print("\nMultidict:")
print(f"\tcapacity: {capacity}")
print("\n")
print(f"\tcost: {cost}")


Multidict:
	capacity: {('Detroit', 'Boston'): 100, ('Detroit', 'New York'): 80, ('Detroit', 'Seattle'): 120, ('Denver', 'Boston'): 120, ('Denver', 'New York'): 120, ('Denver', 'Seattle'): 120}


	cost: {('Detroit', 'Boston'): 7, ('Detroit', 'New York'): 5, ('Detroit', 'Seattle'): 4, ('Denver', 'Boston'): 8, ('Denver', 'New York'): 11, ('Denver', 'Seattle'): 4}


## [环境](https://www.gurobi.com/documentation/current/refman/py_env2.html)

```
+--------------------------------------------+
| 环境                                       |
| +----------------------------------------+ |
| | 模型                                   | |
| | +------+ +-----------+ +-------------+ | | 
| | |    数据   | |   变量   | |   约束   | | |
| | +------+ +-----------+ +-------------+ | |
| | +-----------+                          | |
| | | 目标函数   |                          | |
| | +-----------+                          | |
| +----------------------------------------+ |
+--------------------------------------------+  
```

Python API有一个默认环境，除非创建了一个新环境并显式地传递给需要环境的例程，否则默认情况下使用该环境。

创建环境的主要原因是能够控制你的应用程序何时开始使用 Gurobi 以及何时停止使用它！
- 使用远程资源（如浮动、云或计算服务器许可证）进行优化
- 使用 Jupyter 笔记本时的垃圾回收

注意：最好通过[上下文管理器](https://support.gurobi.com/hc/en-us/articles/4424054948881-How-do-I-manage-Gurobi-environments-in-gurobipy)创建新环境。

## [模型](https://www.gurobi.com/documentation/current/refman/py_model.html)
[Python API](https://www.gurobi.com/documentation/current/refman/py_python_api_overview.html)中的模型构建是面向对象的。参考手册包含模型对象的[完整方法列表](https://www.gurobi.com/documentation/current/refman/py_python_api_details.html#sec:Python-details)。

构造模型对象的语法是：
```
Model(name="", env=defaultEnv)
```

In [5]:
import gurobipy as gp

# 使用默认环境构建模型对象
with gp.Model(name="model") as model:
    pass

model = gp.Model(name="model")
model.dispose()
gp.disposeDefaultEnv()

# 用新环境构建模型对象
with gp.Env() as env, gp.Model(name="model", env=env) as model:
    pass

env = gp.Env()
model = gp.Model(name="model", env=env)
model.dispose()
env.dispose()

Set parameter LicenseID to value 2601452
Freeing default Gurobi environment
Set parameter LicenseID to value 2601452
Set parameter LicenseID to value 2601452


## 决策变量

由于变量与特定的模型对象相关联，因此使用[Model.addVar()](https://www.gurobi.com/documentation/current/refman/py_model_addvar.html#pythonmethod:Model.addVar)方法创建Gurobi变量对象（[Var](https://www.gurobi.com/documentation/current/refman/py_var.html)）：
```
Model.addVar(lb=0, ub=float("inf"), obj=0, vtype=GRB.CONTINUOUS, name="", column=None)
```

在Gurobi中可用的变量类型有:
- 连续: `GRB.CONTINUOUS`
- 一般整数: `GRB.INTEGER`
- 二进制: `GRB.BINARY`
- 半连续: `GRB.SEMICONT`
- 半整数: `GRB.SEMIINT`

半连续变量具有取值为0或介于指定的下界和上界之间的值的性质。半整数变量增加了额外的限制，即变量必须取整数值。

In [ ]:
import gurobipy as gp
from gurobipy import GRB

with gp.Model(name="model") as model:
    # 定义一个二进制变量
    x = model.addVar(vtype=GRB.BINARY, name="x")
    # 定义一个整数变量，下界为-1，上界为100
    y = model.addVar(lb=-1, ub=100, vtype=GRB.INTEGER, name="y")

### [Model.addVars()](https://www.gurobi.com/documentation/current/refman/py_model_addvars.html#pythonmethod:Model.addVar)

要将多个决策变量添加到模型中，请使用Model.addVars()方法，该方法返回一个包含新创建变量的Gurobi tupledict对象：
```
Model.addVars(*indices, lb=0.0, ub=float('inf'), obj=0.0, vtype=GRB.CONTINUOUS, name="")
```
第一个参数是访问变量的索引：
- Integers
- lists of scalars
- tuplelist
- generator

给定的名称下标为生成器表达式的索引。
- 名称以ASCII字符串的形式存储
    - 避免使用包含非ascii字符和空格的名称

In [6]:
import gurobipy as gp
from gurobipy import GRB

with gp.Model(name="model") as model:
    # 二进制变量的三维数组
    x = model.addVars(2, 3, 4, vtype=GRB.BINARY, name="x")
    model.update()
    print(model.getAttr("VarName", model.getVars()))

    # 使用任意的不可变对象列表来创建包含6个变量的元数组
    y = model.addVars([1, 5], [7, 3, 2], ub=range(6), name=[f"y_{i}" for i in range(6)])
    model.update()
    print("\nVariables names, upper bounds, and indices:")
    for index, var in y.items():
        print(f"name: {var.VarName}, ub: {var.UB}, index: {index}")

    # 使用任意的元组列表作为索引
    z = model.addVars(
        [(3, "a"), (3, "b"), (7, "b"), (7, "c")],
        lb=-GRB.INFINITY,
        ub=GRB.INFINITY,
        name="z",
    )
    model.update()
    print("\nVariables names and lower and upper bounds:")
    for index, var in z.items():
        print(f"name: {var.VarName}, lb: {var.LB}, ub: {var.UB}")

Set parameter LicenseID to value 2601452
['x[0,0,0]', 'x[0,0,1]', 'x[0,0,2]', 'x[0,0,3]', 'x[0,1,0]', 'x[0,1,1]', 'x[0,1,2]', 'x[0,1,3]', 'x[0,2,0]', 'x[0,2,1]', 'x[0,2,2]', 'x[0,2,3]', 'x[1,0,0]', 'x[1,0,1]', 'x[1,0,2]', 'x[1,0,3]', 'x[1,1,0]', 'x[1,1,1]', 'x[1,1,2]', 'x[1,1,3]', 'x[1,2,0]', 'x[1,2,1]', 'x[1,2,2]', 'x[1,2,3]']

Variables names, upper bounds, and indices:
name: y_0, ub: 0.0, index: (1, 7)
name: y_1, ub: 1.0, index: (1, 3)
name: y_2, ub: 2.0, index: (1, 2)
name: y_3, ub: 3.0, index: (5, 7)
name: y_4, ub: 4.0, index: (5, 3)
name: y_5, ub: 5.0, index: (5, 2)

Variables names and lower and upper bounds:
name: z[3,a], lb: -inf, ub: inf
name: z[3,b], lb: -inf, ub: inf
name: z[7,b], lb: -inf, ub: inf
name: z[7,c], lb: -inf, ub: inf


## 约束
与变量一样，约束也与模型相关联。使用[Model.addConstr()](https://www.gurobi.com/documentation/current/refman/py_model_addconstr.html)方法向模型添加约束。
```
Model.addConstr(constr, name="")
```

`constr`是一个[TempConstr](https://www.gurobi.com/documentation/current/refman/py_tempconstr.html#pythonclass:TempConstr)对象，可以采用不同的类型：

- 线性约束: `x + y <= 1` 
- 范围线性约束: `x + y == [1, 3]`
- 二次约束: `x*x + y*y + x*y <= 1`
- 线性矩阵约束: `A @ x <= 1`
- 二次矩阵约束: `x @ Q @ y <= 2`
- 绝对值约束: `x == abs_(y)`
- 逻辑约束: `x == and_(y, z)`
- 最小或最大约束: `z == max_(x, y, constant=9)`
- 指标约束: `(x == 1) >> (y + z <= 5)`

In [7]:
import gurobipy as gp
from gurobipy import GRB

# 为任意给定的n和b添加约束“\sum_{i=1}^{n} x_i <= b”。假设x_i是二进制变量
n, b = 10, 4
with gp.Model("model") as model:
    x = model.addVars(n, vtype=GRB.BINARY, name="x")
    model.addConstr(gp.quicksum(x[i] for i in range(n)) <= b, name="c1")
    model.update()

    # 打印约束c1的LHS、Sense和RHS
    c1 = model.getConstrByName("c1")
    print(f"RHS, sense = {c1.RHS}, {c1.Sense}")
    print(f"row: {model.getRow(c1)}")
    print("\n\n")

# 添加约束“x_i y_j - x_i*y_j >= 3”。假设x_i和y_j是连续的
n, m = 5, 4
with gp.Model("model") as model:
    x = model.addVars(n, name="x")
    y = model.addVars(m, name="y")

    for i in range(n):
        for j in range(m):
            model.addConstr(x[i] + y[j] - x[i] * y[j] >= 3, name=f"c_{i}{j}")

    model.update()

    # Print the LHS, Sense, and RHS of all c_ij constraints
    for c in model.getQConstrs():
        print(f"Name: {c.QCName}")
        print(f"\tRHS, sense = {c.QCRHS}, {c.QCSense}")
        print(f"\trow: {model.getQCRow(c)}")

RHS, sense = 4.0, <
row: x[0] + x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9]



Name: c_00
	RHS, sense = 3.0, >
	row: x[0] + y[0] + [ -1.0 x[0] * y[0] ]
Name: c_01
	RHS, sense = 3.0, >
	row: x[0] + y[1] + [ -1.0 x[0] * y[1] ]
Name: c_02
	RHS, sense = 3.0, >
	row: x[0] + y[2] + [ -1.0 x[0] * y[2] ]
Name: c_03
	RHS, sense = 3.0, >
	row: x[0] + y[3] + [ -1.0 x[0] * y[3] ]
Name: c_10
	RHS, sense = 3.0, >
	row: x[1] + y[0] + [ -1.0 x[1] * y[0] ]
Name: c_11
	RHS, sense = 3.0, >
	row: x[1] + y[1] + [ -1.0 x[1] * y[1] ]
Name: c_12
	RHS, sense = 3.0, >
	row: x[1] + y[2] + [ -1.0 x[1] * y[2] ]
Name: c_13
	RHS, sense = 3.0, >
	row: x[1] + y[3] + [ -1.0 x[1] * y[3] ]
Name: c_20
	RHS, sense = 3.0, >
	row: x[2] + y[0] + [ -1.0 x[2] * y[0] ]
Name: c_21
	RHS, sense = 3.0, >
	row: x[2] + y[1] + [ -1.0 x[2] * y[1] ]
Name: c_22
	RHS, sense = 3.0, >
	row: x[2] + y[2] + [ -1.0 x[2] * y[2] ]
Name: c_23
	RHS, sense = 3.0, >
	row: x[2] + y[3] + [ -1.0 x[2] * y[3] ]
Name: c_30
	RHS, sense = 3.0

### [Model.addConstrs](https://www.gurobi.com/documentation/current/refman/py_model_addconstrs.html)

要向模型中添加多个约束，使用model.addconstrs()方法，该方法返回一个包含新创建约束的Gurobi tupledict：

```
Model.addConstrs(generator, name="")
```

In [8]:
import gurobipy as gp
from gurobipy import GRB

# 对所有（i, j）添加约束x_i + y_j <= 1。假设x_i和y_j是二进制变量
I = range(5)
J = ["a", "b", "c"]
with gp.Model("model") as model:
    x = model.addVars(I, vtype=GRB.BINARY, name="x")
    y = model.addVars(J, vtype=GRB.BINARY, name="y")

    generator = (x[i] + y[j] <= 1 for i in I for j in J)
    model.addConstrs(generator, name="c")
    model.update()

    # Print constraint names
    print(model.getAttr("ConstrName", model.getConstrs()))

['c[0,a]', 'c[0,b]', 'c[0,c]', 'c[1,a]', 'c[1,b]', 'c[1,c]', 'c[2,a]', 'c[2,b]', 'c[2,c]', 'c[3,a]', 'c[3,b]', 'c[3,c]', 'c[4,a]', 'c[4,b]', 'c[4,c]']


## 目标函数

要将模型目标设置为线性或二次表达式，请使用[Model.setObjective()](https://www.gurobi.com/documentation/current/refman/py_model_setobjective.html)方法：
```
Model.setObjective(expr, sense=GRB.MINIMIZE)
```
- expr: 
    - [LinExpr()](https://www.gurobi.com/documentation/current/refman/py_lex.html): 常数加上系数-变量对捕获线性项
    - [QuadExpr()](https://www.gurobi.com/documentation/current/refman/py_qex.html): 线性表达式+系数-变量-变量三元组列表
- sense:
    - GRB.MINIMIZE (default) or GRB.MAXIMIZE

In [9]:
import gurobipy as gp
from gurobipy import GRB

import numpy as np

# 添加形如c^Tx的线性目标和形如x^TQx的二次目标。
n = 10
c = np.random.rand(n)
Q = np.random.rand(n, n)

with gp.Model("model") as model:
    x = model.addVars(n, name="x")
    linexpr = gp.quicksum(c[i] * x[i] for i in range(n))
    # linexpr = gp.quicksum(c_i * x_i for c_i, x_i in zip(c, x.values()))
    model.setObjective(linexpr)
    model.update()

    # 打印目标表达式
    obj = model.getObjective()
    print(f"obj: {obj}")

with gp.Model("model") as model:
    x = model.addVars(n, name="x")
    quadexpr = 0
    for i in range(n):
        for j in range(n):
            quadexpr += x[i] * Q[i, j] * x[j]
    model.setObjective(quadexpr)
    model.update()

    # Print objective expression
    obj = model.getObjective()
    print(f"\nobj: {obj}")

obj: 0.1512515445081677 x[0] + 0.3291541476595792 x[1] + 0.8005249778673674 x[2] + 0.974099989221955 x[3] + 0.2628186839588105 x[4] + 0.9070584360180418 x[5] + 0.9627486894384101 x[6] + 0.1280724390723441 x[7] + 0.6882885209849768 x[8] + 0.932295625697586 x[9]

obj: 0.0 + [ 0.7525031342511642 x[0] ^ 2 + 1.143852823313597 x[0] * x[1] + 1.3055475113378765 x[0] * x[2] + 0.7634217626453419 x[0] * x[3] + 0.7285503698021777 x[0] * x[4] + 1.2889420517441725 x[0] * x[5] + 1.1886704269940966 x[0] * x[6] + 1.4002153014202987 x[0] * x[7] + 1.0989281251875473 x[0] * x[8] + 0.3372517801257665 x[0] * x[9] + 0.698134085527861 x[1] ^ 2 + 1.0248325349620095 x[1] * x[2] + 0.9786676308573096 x[1] * x[3] + 0.9696252081631695 x[1] * x[4] + 1.3858910396039117 x[1] * x[5] + 0.7344468364738388 x[1] * x[6] + 0.8731192560967127 x[1] * x[7] + 0.8384893144593442 x[1] * x[8] + 1.1184807470441274 x[1] * x[9] + 0.21993985560974383 x[2] ^ 2 + 1.136977729838955 x[2] * x[3] + 1.1115908885695744 x[2] * x[4] + 1.85846404

## [属性](https://www.gurobi.com/documentation/current/refman/attributes.html)

查询和修改Gurobi对象属性的主要机制是通过属性接口。您可以在上面链接的参考中看到完整的Gurobi属性集。

让我们看一个如何在优化完成后查询模型对象上有用属性的示例。

In [ ]:
import urllib.request
import gurobipy as gp
from gurobipy import GRB

url = "https://raw.githubusercontent.com/Gurobi/modeling-examples/master/intro_to_gurobipy/data/glass4.mps.bz2"
path_to_file, _ = urllib.request.urlretrieve(url, "./glass4.mps.bz2")

# 本地运行笔记本:
# with gp.read("./data/glass4.mps.bz2") as model:
with gp.read(path_to_file) as model:
    model.optimize()

    print("****************** SOLUTION ******************")
    print(f"\tStatus       : {model.Status}")
    print(f"\tObj          : {model.ObjVal}")
    print(f"\tSolutionCount: {model.SolCount}")
    print(f"\tRuntime      : {model.Runtime}")
    print(f"\tMIPGap       : {model.MIPGap}")

    print("\n")
    for var in model.getVars()[:20]:
        print(f"\t{var.VarName} = {var.X}")

RemoteDisconnected: Remote end closed connection without response

## [参数](https://www.gurobi.com/documentation/current/refman/parameter_descriptions.html)
参数控制着Gurobi Optimizer的机制。

In [ ]:
import urllib.request
import gurobipy as gp
from gurobipy import GRB

url = "https://raw.githubusercontent.com/Gurobi/modeling-examples/master/intro_to_gurobipy/data/glass4.mps.bz2"
path_to_file, _ = urllib.request.urlretrieve(url, "./glass4.mps.bz2")

# 本地运行笔记本:
# with gp.read("data/glass4.mps.bz2") as model:
with gp.read(path_to_file) as model:
    model.params.Threads = 1
    model.params.TimeLimit = 10
    model.optimize()

## [一般的约束](https://www.gurobi.com/documentation/current/refman/constraints.html#subsubsection:GeneralConstraints)

一般约束允许很容易地定义某些变量关系，而不需要根据更基本的MIP约束对这些关系进行建模。捕获这些一般约束中的单个约束通常需要大量约束和许多辅助决策变量。

- __简单一般约束__: 
    - $z = \mbox{max(x, y, 3)}$: `model.addConstr(z == max_(x, y, constant=3))`
    - $z = \mbox{min(x, y, 3)}$: `model.addConstr(z == min_(x, y, constant=3))`
    - $y = \mbox{abs(x)}$: `model.addConstr(y == abs_(x))`
    - $z = x \land y$: `model.addConstr(z == and_(x, y))`
    - $z = x \lor y$: `model.addConstr(z == or_(x, y))`
    - $z = ||x||_p, ~~ p = 0, 1, 2, \infty$: `model.addConstr(nx == norm(x, 1.0))`
    - indicator: 
        -  $x_0 = 1 -> x_1 + 2 x_2 + x_3 \leq 1$
            - `model.addGenConstrIndicator(x0, 1, x1 + 2*x2 + x3 <= 1)`
            - `model.addConstr((x0 == 1) >> (x1 + 2*x2 + x3 <= 1))`
    - piece-wise linear:
        - `model.addGenConstrPWL(x, y, [0, 1, 2], [1.5, 0, 3], "")`
- __函数的约束__: 
    - $y = p_0x^n + p_1x^{n-1} + \ldots + p_nx+ p_{n+1}$:
        - $y = 2 x^3 + 1.5 x^2 + 1$
          - `model.addGenConstrPoly(x, y, [2, 1.5, 0, 1])`
    - $y = e^x$: `model.addGenConstrExp(x, y)`
    - $y = a^x$: `model.addGenConstrExpA(x, y, a)`
    - $y = \ln(x)$: `model.addGenConstrLog(x, y)`
    - $y = \log_a(x)$: `model.addGenConstrLogA(x, y, a)`
    - $y = \frac{1}{1+e^{-x}}$: `model.addGenConstrLogistic(x, y)`
    - $y = x^a$: `model.addGenConstrPow(x, y, a)`
    - $y = \sin(x)$: `model.addGenConstrSin(x, y)`
    - $y = \cos(x)$: `model.addGenConstrCos(x, y)`
    - $y = \tan(x)$: `model.addGenConstrTan(x, y)`
   
Gurobi将自动向模型中添加函数的分段线性近似。

In [ ]:
# 考虑下面的非凸非线性问题
#
#  maximize    2 x    + y
#  subject to  exp(x) + 4 sqrt(y) <= 9
#              x, y >= 0

import gurobipy as gp
from gurobipy import GRB
import math

with gp.Model("model") as model:
    x = model.addVar(name="x")
    y = model.addVar(name="y")
    u = model.addVar(name="u")
    v = model.addVar(name="v")

    # Set objective
    model.setObjective(2 * x + y, GRB.MAXIMIZE)

    # u = exp(x)
    gcf1 = model.addGenConstrExp(x, u, name="gcf1")
    # v = y^(0.5)
    gcf2 = model.addGenConstrPow(y, v, 0.5, name="gcf2")
    c = model.addConstr(u + 4 * v <= 9)

    # 使用等长方法，长度= 1e-3
    model.Params.FuncPieces = 1
    model.Params.FuncPieceLength = 1e-3

    # 优化模型
    model.optimize()

    print("****************** SOLUTION ******************")
    print(f"x = {x.X}, u = {u.X}")
    print(f"y = {y.X}, v = {v.X}")
    print(f"Obj = {model.ObjVal}")

    # 计算 exp(x)+4sqrt(y)<=9 的违反情况
    vio = math.exp(x.X) + 4 * math.sqrt(y.X) - 9
    if vio < 0:
        vio = 0
    print(f"Vio = {vio}")

Set parameter FuncPieces to value 1
Set parameter FuncPieceLength to value 0.001
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Non-default parameters:
FuncPieces  1
FuncPieceLength  0.001

Optimize a model with 1 rows, 4 columns and 2 nonzeros
Model fingerprint: 0x1d1d047b
Model has 2 function constraints treated as nonlinear
  1 EXP, 1 POW
Variable types: 4 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+00, 9e+00]
Presolve time: 0.01s
Presolved: 11 rows, 5 columns, 22 nonzeros
Presolved model has 2 nonlinear constraint(s)

Solving non-convex MINLP

Variable types: 5 continuous, 0 integer (0 binary)
Found heuristic solution: objective 4.4735963

Root relaxation: ob

## 示例：投资组合优化

在投资组合优化问题中，有$n$资产。每个资产$i$与一个预期收益$\mu_i$相关联，每一对资产$(i, j)$具有一个协方差（风险）$ \sigma_{ij}$。目标是找到投资组合中投资于每种资产的最优比例，以使投资风险最小化，从而使1)投资的总预期回报超过最小目标回报$\mu_0$和2)投资组合最多投资$k \leq n$资产。

- $x_i$: 相对投资于资产$i$
- $y_i$: $i$是否被交易的二元变量

\begin{align}
\mbox{minimize} \quad & \sum_{i=1}^{n} \sum_{j=1}^{n} \sigma_{ij} x_i x_j & \notag \\
\mbox{subject to} \quad & \sum_{i=1}^{n} \mu_i x_i \geq \mu_0 & \notag \\
                        & \sum_{i=1}^{n} x_i = 1 & \notag \\
                        & \sum_{i=1}^{n} y_i \leq k & \notag \\
                        & x_i \leq y_i & i=1, \ldots, n \notag \\
                        & 0 \leq x_i \leq 1 & i=1, \ldots, n \notag \\
                        & y_i \in \{0, 1\} & i=1, \ldots, n \notag
\end{align}

In [15]:
import json
from urllib.request import urlopen
import numpy as np

url = "https://raw.githubusercontent.com/Gurobi/modeling-examples/master/intro_to_gurobipy/data/portfolio-example.json"
response = urlopen(url)
data = json.loads(response.read())

n = data["num_assets"]
sigma = np.array(data["covariance"])
mu = np.array(data["expected_return"])
mu_0 = data["target_return"]
k = data["portfolio_max_size"]

In [16]:
import gurobipy as gp
from gurobipy import GRB

# 将数据转换为DataFrame
import pandas as pd

with gp.Model("term-based") as model:
    x = model.addVars(n, ub=1, name="x")
    y = model.addVars(n, vtype=GRB.BINARY, name="y")

    risk = gp.quicksum(x[i] * sigma[i, j] * x[j] for i in range(n) for j in range(n))
    # 另一种构建风险表达的方法
    # risk = gp.QuadExpr()
    # for i in range(n):
    #    for j in range(n):
    #        risk.addTerms(sigma[i, j], x[i], x[j])
    model.setObjective(risk)

    expected_return = gp.quicksum(mu[i] * x[i] for i in range(n))
    model.addConstr(expected_return >= mu_0, name="return")

    model.addConstr(x.sum() == 1, name="budget")
    # 另一种构建预算约束的方法
    # model.addConstr(gp.quicksum(x[i] for i in range(n)) == 1, name="budget")

    model.addConstr(y.sum() <= k, name="cardinality")

    model.addConstrs((x[i] <= y[i] for i in range(n)), name="is_allocated")

    model.optimize()

    # 将结果转换为DataFrame
    portfolio = [var.X for var in model.getVars() if "x" in var.VarName]
    risk = model.ObjVal
    expected_return = model.getRow(model.getConstrByName("return")).getValue()
    df = pd.DataFrame(
        data=portfolio + [risk, expected_return],
        index=[f"asset_{i}" for i in range(n)] + ["risk", "return"],
        columns=["Portfolio"],
    )

    print(df)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 23 rows, 40 columns and 100 nonzeros
Model fingerprint: 0x755216b2
Model has 210 quadratic objective terms
Variable types: 20 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [7e-06, 4e-03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-04, 2e+01]
Found heuristic solution: objective 0.0000960
Presolve time: 0.01s
Presolved: 23 rows, 40 columns, 100 nonzeros
Presolved model has 210 quadratic objective terms
Variable types: 20 continuous, 20 integer (20 binary)

Root relaxation: objective 6.741224e-05, 38 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl U

## 最佳实践

- 确保模型和数据之间的分离
- 在适当的情况下使用描述性名称
- 创建变量/约束时利用稀疏性-仅为有效组合创建变量/约束
- 不要忘记处理您的模型和环境
- 使用留档

## [卡牌游戏](https://www.gurobi.com/resources/optimization-gamification-introducing-the-gurobipy-card-game/)


<img src="./images/card-game.png" width="800" height="600" style="margin-left:auto; margin-right:auto"/>


## 资源
- [Gurobi Python Documentation](https://www.gurobi.com/documentation/current/refman/py_python_api_overview.html)
- [Gurobi Python Examples](https://www.gurobi.com/documentation/current/examples/python_examples.html)
- [Gurobi Jupyter Notebook Modeling Examples](https://www.gurobi.com/jupyter_models/)
- [Gurobi Knowledge Base](https://support.gurobi.com/hc/en-us/categories/360000840331-Knowledge-Base)